In [1]:
"""
HOAC - Higher-Order Ambisonics Audio Compression Codec

@author: Chris Hold
"""
from pathlib import Path
import time
import numpy as np

import spaudiopy as spa

import hoac

from IPython.display import Audio


In [2]:
print("HOAC v:", hoac.get_version())
print("libopus v:", hoac.pylibopus.api.info.get_version_string())

HOAC v: 0.2
libopus v: b'libopus 1.5.2'


In [3]:
#input
fs = 48000
sig_len = 10  # in s
N_sph_in = 5

file_name = 'MachineAer_snip_ambix5.wav'
in_path = Path('./audio') / Path(file_name)

# name of output
hoac_file = Path("./out_file.hoac")

In [4]:
# load signal
in_file = spa.io.load_audio(in_path, fs)
assert in_file.fs == fs
in_file.trim(0, None, sig_len)
sig_in = in_file.get_signals()[:(N_sph_in+1)**2, :]
if any(s in str(in_path).lower() for s in ['ambix', 'sn3d']):
    sig_in = spa.sph.sn3d_to_n3d(sig_in)
    print("Converted SN3D input")

Converted SN3D input


/home/chris/git/spaudiopy/spaudiopy/sig.py:186: UserWarning: Stop exceeds signal.
  warn("Stop exceeds signal.")


In [5]:
# Initialize encoder
hoac_encoder = hoac.Encoder(fs=fs, profile='high')
hoac_encoder.prepare()

In [6]:
# Initialize and start timer
start_time = time.time()

print("HOAC encoding")

# Encode
data_pars_status, data_tcs, data_pars_stream = hoac_encoder.encode(sig_in)

# write file
hoac.write_hoac(data_pars_status, data_tcs, data_pars_stream, hoac_file)

print('Writing output: ', round(time.time()-start_time, 3), 'seconds.')
print(f'Filesize: {hoac_file.stat().st_size/10e5} MB')



HOAC encoding
Writing output:  2.352 seconds.
Filesize: 0.321242 MB


In [7]:
# read file
conf, c_data_tcs, c_data_pars = hoac.read_hoac(hoac_file)

# initialize decoder
hoac_decoder = hoac.Decoder(fs=fs, N_sph_out=N_sph_in)
hoac_decoder.prepare(conf=conf)


In [8]:
# Initialize and start timer
start_time = time.time()

print("HOAC decoding")
sig_out = hoac_decoder.decode(c_data_tcs, c_data_pars)

print('Decoding: ', round(time.time()-start_time, 3), 'seconds.')


HOAC decoding
Decoding Audio
Decoding:  2.259 seconds.


In [9]:
# Compare
Path('./audio').mkdir(parents=True, exist_ok=True)
spa.io.save_audio(spa.sph.n3d_to_sn3d(sig_in).T,
                  './audio/in_sig_ambix.wav', fs)
spa.io.save_audio(spa.sph.n3d_to_sn3d(sig_out).T,
                  "./audio/out_hoac_ambix.wav", fs)

In [10]:
hrirs = spa.io.load_hrirs(fs)
hrirs_nm = spa.decoder.magls_bin(hrirs, N_sph_in)


In [11]:
in_bin = spa.sig.MultiSignal([*spa.decoder.sh2bin(0.5*sig_in, hrirs_nm)],
                             fs=fs)
Audio(in_bin, rate=fs, normalize=False)

In [12]:
out_bin = spa.sig.MultiSignal([*spa.decoder.sh2bin(0.5*sig_out, hrirs_nm)],
                              fs=fs)
Audio(out_bin, rate=fs, normalize=False)